### Optimal Portfolio Allocation

An investment universe of the following risky assets with a dependence structure (correlation) is given:

\begin{equation*}

\begin{matrix}
\textbf{Asset} & \boldsymbol\mu & \boldsymbol\sigma & \boldsymbol\omega\\
A & 0.02 & 0.05 & \omega_1\\
B & 0.07 & 0.12 & \omega_2\\
C & 0.15 & 0.17 & \omega_3\\
D & 0.20 & 0.25 & \omega_4
\end{matrix}
\qquad
\qquad
R =
\begin{pmatrix}
1 & 0.3 & 0.3 & 0.3\\
0.3 & 1 & 0.6 & 0.6\\
0.3 & 0.6 & 1 & 0.6\\
0.3 & 0.6 & 0.6 & 1
\end{pmatrix}

\end{equation*}

**Question 2.** Consider optimisation for a tangency portfolio (maximum Sharpe Ratio). 

- Formulate optimisation expression. 
- Formulate Lagrangian function and give its partial derivatives only. 
- For the range of tangency portfolios given by $r_f = 50bps, 100bps, 150bps, 175bps$ optimal compute allocations (ready formula) and $\sigma_\Pi$ . <ins>Present results in a table</ins>. 
    - Plot the efficient frontier in the presence of a risk-free asset for $r_f = 100bps, 175bps$

### Answers and code

The tangency portfolio is invested wholly in risky assets and that maximises the sharpe ratio.

\begin{equation*}
\underset{\boldsymbol\omega}{\arg\max} \quad \frac{\mu - r_f}{\sigma} = \frac{\omega' \mu - r_f}{(\omega' \Sigma \omega)^{\frac{1}{2}}}
\qquad
s.t.
\quad
\omega'1 = 1
\end{equation*}

Form the Langrange function with two multiplier $\lambda$ for a given vector of weights $w$
\begin{equation*}
L(w, \lambda) = (w' \mu - r_f)(w' \Sigma w)^{-\frac{1}{2}} + \lambda(1 - w'1)
\end{equation*}

##### Derivatives

Using a combination of the chain and product rules to obtain the first order derivative

$$
\frac{\partial L}{\partial w} = \mu (w' \Sigma w)^{-\frac{1}{2}} - \frac{1}{2}(w' \mu - r_f)(w' \Sigma w)^{-\frac{3}{2}}2 \Sigma w + \lambda(-1)
$$
simplifying 
$$
\frac{\partial L}{\partial w} = \mu (w' \Sigma w)^{-\frac{1}{2}} - (w' \mu - r_f)(w' \Sigma w)^{-\frac{3}{2}} \Sigma w - \lambda
$$

and the second order derivative

$$
\frac{\partial^2 L}{\partial w^2} = -\frac{1}{2}\mu (w' \Sigma w)^{-\frac{5}{2}}  2\Sigma w - -\frac{3}{2} (w'\Sigma w)^{-frac{5}{2}} 2 \Sigma w \Sigma w + (w' \Sigma w)^{-\frac{3}{2}} \Sigma
$$

simplifying

$$
\frac{\partial^2 L}{\partial w^2} = -\mu (w' \Sigma w)^{-\frac{5}{2}}  \Sigma w + 3 (w'\Sigma w)^{-\frac{5}{2}}  \Sigma w \Sigma w - (w' \Sigma w)^{-\frac{3}{2}} \Sigma
$$

The ready formula to calculate the optimal weights of the tangency portfolio is given by

$$
w_t = \frac{\Sigma^{-1} (\mu - r_f1)}{B - Ar_f}
$$

where A and B are scalars as previously defined in question 1

\begin{equation*}
A = 1' \Sigma^{-1} 1
\end{equation*}

\begin{equation*}
B = \mu' \Sigma^{-1} 1 = 1' \Sigma^{-1} \mu
\end{equation*}

In [2]:
# Import the relevant packages
# Import numpy
import plotly.express as px
import numpy as np
from numpy import *
from numpy.linalg import multi_dot, inv

# Import pandas
import pandas as pd

# Import plotly express
px.defaults.template = 'ggplot2'
px.defaults.width, px.defaults.height = 650, 450
import plotly.io as pio
pio.renderers.default='notebook'

In [3]:
# Set the risk free rates
r = [0.005, 0.01, 0.015, 0.0175]

# Set the matrices for mu and sigma and 1
mu = np.array([0.02, 0.07, 0.15, 0.2]).reshape((4, 1))
sigma = np.array([0.05, 0.12, 0.17, 0.25]).reshape((4, 1))
R = np.array([[1, 0.3, 0.3, 0.3], [0.3, 1, 0.6, 0.6], [
             0.3, 0.6, 1, 0.6], [0.3, 0.6, 0.6, 1]]).reshape(4, 4)
one = np.ones((4,1))

# Compute the covariance matrix usings SRS where S = the diagonal standard deviation matrix
S = np.diag(sigma.flatten())
covariance_matrix = multi_dot([S, R, S])

# Computer the inverse covariance matrix
inverse_covariance_matrix = inv(covariance_matrix)

# Compute A and B
A = multi_dot((one.T,inverse_covariance_matrix,one))
B = multi_dot((one.T,inverse_covariance_matrix,mu))

In [16]:
# For each risk free rate, compute the optimal weights and add them to a dataframe
arr = []
for rate in r:
    risk_premium = mu - multi_dot((rate, one))
    weights = (multi_dot((inverse_covariance_matrix, risk_premium)) /
               (B - A*rate))
    # Add the risk free rate and the sigma_pi
    sigma_pi = multi_dot((weights.T, covariance_matrix, weights))
    row = np.insert(weights, 0, rate)
    row = np.append(row, sigma_pi)
    arr.append(row)

df = pd.DataFrame(arr, columns=['rf', 'w_1', 'w_2', 'w_3', 'w_4', 'sigma_pi'])
df


,rf,w_1,w_2,w_3,w_4,sigma_pi
0,0.0050,0.016835,-0.229367,0.814340,0.398192,0.038616
1,0.0100,-0.745937,-0.510569,1.490249,0.766257,0.122966
2,0.0150,-8.644854,-3.422571,8.489651,4.577774,3.890329
3,0.0175,8.103502,2.751851,-6.351431,-3.503922,2.171246


The efficient frontier with N risky assets and the risk free asset will have the below return and volatility

$$
\mu_\Pi = r_f + w'(\mu - r_f 1)
$$
$$
\sigma_\Pi = w' \Sigma w
$$

In [32]:
# Set values for N and initiate an empty list for the
N = 1000
weights_arr = []

# Generate N random weights for the 4 assets
for i in range(N):
    weights = random.uniform(-1,1,4)
    weights_arr.append(weights)

# Add them to a dataframe
df = pd.DataFrame(weights_arr, columns=['w_1', 'w_2', 'w_3', 'w_4'])

# loop through weights, extract matrix, calculate portfolio mean and sd, append back to df
for index, row in df.iterrows():
    # convert to a matrix of weights
    w = np.array(row).reshape(1, 4)
    # Compute mean for the portfolio, add it to the dataframe
    mu_pi = multi_dot([w, mu])
    df.loc[index, 'mu_pi'] = mu_pi.flatten()
    # Compute the sd for the portfolio, add it to the dataframe
    sigma_pi = np.sqrt(multi_dot([w, covariance_matrix, w.T]))
    df.loc[index, 'sigma_pi'] = sigma_pi.flatten()

In [33]:
df.head()

,w_1,w_2,w_3,w_4,mu_pi,sigma_pi
0,0.202536,0.023001,-0.877802,0.898089,0.053608,0.182502
1,-0.399972,-0.268188,-0.587072,-0.252112,-0.165256,0.177610
2,-0.094435,-0.734777,-0.148230,-0.907860,-0.257130,0.307332
3,-0.031754,0.286771,-0.265552,-0.376192,-0.095632,0.107315
4,0.160279,0.380634,0.046802,0.144742,0.065818,0.081976


In [34]:
# Plotting the results
fig = px.scatter(df, x='sigma_pi', y='mu_pi', color='mu_pi', color_continuous_scale='Viridis',
                 labels={'sigma_pi': 'Expected Volatility',
                         'mu_pi': 'Expected Return'},
                 title='Efficient Frontier highlighting maximum and minimum return')
idx_max_mu = df['mu_pi'].idxmax()
idx_min_sigma = df['sigma_pi'].idxmin()

fig.add_scatter(x=[df['sigma_pi'].loc[idx_max_mu]], y=[df['mu_pi'].loc[idx_max_mu]], marker=dict(color='green', size=10, symbol='cross'),
                name='Maximum Return').update(layout_showlegend=False)
fig.add_scatter(x=[df['sigma_pi'].loc[idx_min_sigma]], y=[df['mu_pi'].loc[idx_min_sigma]], marker=dict(color='red', size=10, symbol='cross'),
                name='Minimum Volatility').update(layout_showlegend=False)

fig.show()